# 通过CNN分析价格曲线，预测涨跌  
短， 中， 长线  
20 x 20  20个月 80个礼拜  => if 未来1个月的股价 > 当前股价


In [54]:
import baostock as bs
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
import matplotlib. pyplot as plt 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Reshape,Dropout,Activation
from tensorflow.keras.layers import Conv2D,MaxPooling2D
from tensorflow.keras.layers import Conv1D,MaxPooling1D
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split

In [3]:
# 读取csv文件
stock_code = "300721.SZ"
stock = pd.read_csv(stock_code + ".csv")

In [26]:
predict_period = 20 
history_period = 400

In [17]:
stock['future_price'] = stock['close'].rolling(predict_period).mean()

In [18]:
stock

,date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,peTTM,pbMRQ,psTTM,pcfNcfTTM,isST,future_price
0,2017-11-15,sz.300721,20.05,24.06,20.05,24.06,16.71,8575,205713.50,3,0.042768,1,43.985640,30.657271,3.573392,1.658425,-15.129562,0,NaN
1,2017-11-16,sz.300721,26.47,26.47,26.47,26.47,24.06,2640,69880.80,3,0.013167,1,10.016620,33.728095,3.931326,1.824543,-16.645034,0,NaN
2,2017-11-17,sz.300721,29.12,29.12,29.12,29.12,26.47,3200,93184.00,3,0.015960,1,10.011340,37.104728,4.324904,2.007204,-18.311424,0,NaN
3,2017-11-20,sz.300721,32.03,32.03,32.03,32.03,29.12,6625,212198.75,3,0.033042,1,9.993125,40.812652,4.757097,2.207786,-20.141308,0,NaN
4,2017-11-21,sz.300721,35.23,35.23,35.23,35.23,32.03,22158,780626.34,3,0.110514,1,9.990636,44.890095,5.232361,2.428358,-22.153553,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
800,2021-03-03,sz.300721,36.85,37.23,36.30,36.99,37.13,1185380,43522407.60,3,1.493900,1,-0.377100,-404.037624,3.806287,3.251413,56.558233,0,33.0705
801,2021-03-04,sz.300721,36.90,37.36,36.10,36.30,36.99,1243800,45556710.72,3,1.567500,1,-1.865400,-396.500831,3.735286,3.190762,55.503213,0,33.3240
802,2021-03-05,sz.300721,35.96,36.75,35.90,36.28,36.30,926368,33786622.04,3,1.167400,1,-0.055100,-396.282373,3.733228,3.189004,55.472633,0,33.5690
803,2021-03-08,sz.300721,36.50,37.26,35.69,35.69,36.28,1316400,48238437.00,3,1.659000,1,-1.626200,-389.837870,3.672517,3.137143,54.570515,0,33.7480


In [51]:
# rolling 是向上卷的。 
y = stock['future_price'][history_period-1:].values[:-predict_period]

In [62]:
n = len(stock)
x = np.array([stock['close'][i:i+history_period] for i in range(n-history_period+1)])[20:].reshape([386, 20, 20])[0]

In [63]:
n, x.shape, y.shape

(805, (20, 20), (386,))

In [64]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,shuffle=False)  #拆分数据集，将test集为0.2，参数不同
print(x_train.shape)

plt.plot(y)
plt.plot(y_train)
plt.plot(y_test)

ValueError: Found input variables with inconsistent numbers of samples: [20, 386]

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(20, 20, 1)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(len(train_ds.class_names)))

# model.summary()

model.compile(optimizer='adam', 
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

model.fit(train_ds, validation_data=val_ds, epochs=1)
tf.saved_model.save(model, 'saved_model/')